<a href="https://colab.research.google.com/github/detsutut/bedrock-inference/blob/main/bedrock_inference_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bedrock-inference package: how to use


### 0. Install the bedrock-inference package from Pip

In [ ]:
!pip install bedrock-inference

### 1. Establish a MFA authenticated session

Here, secrets are retrieved from the Colab Secrets vault.

In [6]:
from google.colab import userdata
from bedrock_inference import bedrock

session = bedrock.aws_login_mfa(token="087914",
                                arn=userdata.get('AWS_ARN_MFA_DEVICE'),
                                aws_access_key_id=userdata.get('AWS_ACCESS_KEY_ID'),
                                aws_secret_access_key=userdata.get('AWS_SECRET_ACCESS_KEY'))

Check that the Bedrock service is available in the established session

In [13]:
assert 'bedrock' in session.get_available_services()
assert 'bedrock-runtime' in session.get_available_services()

### 2. Instantiate the Bedrock caller

In [14]:
caller = bedrock.Bedrock(session=session, region="us-west-2")

Check the list of available models for your region and retrieve the target model id.
In this case, we want to use Anthropic's Claude 3 Sonnet, but we don't know the correspondent AWS modelId.

In [36]:
models_df = caller.list_models()
models_df[models_df["modelName"].str.contains('claude 3 sonnet', case=False)]

,modelArn,modelId,modelName,providerName,inputModalities,outputModalities,responseStreamingSupported,customizationsSupported,inferenceTypesSupported,modelLifecycle
18,arn:aws:bedrock:us-west-2::foundation-model/an...,anthropic.claude-3-sonnet-20240229-v1:0,Claude 3 Sonnet,Anthropic,"[TEXT, IMAGE]",[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}


### 3. Invoke the target model and get the generated answer

In [39]:
answer = caller.invoke_model(prompt="What's the conceptual opposite of 'Hello World'?",
                             model_id="anthropic.claude-3-sonnet-20240229-v1:0",
                             maxTokens = 200)
print(answer)

The conceptual opposite of the famous "Hello World" program, which is often used to introduce programming concepts and verify that a language or environment is set up correctly, could be considered "Goodbye World."

"Hello World" is typically the first program that new programmers write, as it serves as a simple introduction to printing output to the console or displaying a message. It's a way of saying "hello" to the world of programming.

On the other hand, "Goodbye World" could symbolize the end of a programming journey, a farewell to the world of coding, or the termination of a program's execution. It's the conceptual antithesis of "Hello World," representing the opposite of starting something new and instead signifying the end or completion of a process.

While "Hello World" is a warm greeting and a starting point, "Goodbye World" could be seen as a final message or a symbolic way to conclude a program or a programming


---------------------

## Advanced Usage

`bedrock-inference` handles API requests for you in a nice and user-friendly way.

However, this is just a nice wrapping of the original API requests.

If you want to take control of what's happening under the hood, you can use the original Bedrock clients to send custom requests.

In the following example, we want to retrieve the raw request output instead of just the textual answer generated by the model.

Check the [boto3 documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime.html) for further details.


In [40]:
client = caller._runtime_client_

In [42]:
response = client.converse(
    modelId="anthropic.claude-3-sonnet-20240229-v1:0",
    messages=[
            {
                "role": "user",
                "content": [{"text": "What's the conceptual opposite of 'Hello World'?"}],
            }
        ],
    inferenceConfig={
        'maxTokens': 200,
    }
)

In [43]:
response

{'ResponseMetadata': {'RequestId': 'c801c2ed-d475-4ef1-a8d4-640d8a0cae9b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 30 Sep 2024 15:50:31 GMT',
   'content-type': 'application/json',
   'content-length': '1213',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c801c2ed-d475-4ef1-a8d4-640d8a0cae9b'},
  'RetryAttempts': 0},
 'output': {'message': {'role': 'assistant',
   'content': [{'text': 'The conceptual opposite of the famous "Hello World" program, which is often used as a simple introduction to programming, could be considered "Goodbye World."\n\n"Hello World" is typically one of the first programs a beginner programmer writes when learning a new programming language. It\'s a simple program that outputs the text "Hello World" to the console or user interface, serving as a way to verify that the development environment is set up correctly and the programmer can successfully compile and run basic code.\n\nIn contrast, "Goodbye World" could be seen as a symbolic re